## Importing libraries

In [820]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Reading input from file

In [821]:
file = open('trainNN.txt')

lines = file.readlines()

X = []
labels = []

for line in lines:
    var = [float(x) for x in line.split()]
    X.append(var[:-1])
    labels.append(int(var[-1]))
    
number_of_classes = max(labels)
N = len(X)

Y = np.zeros((N,number_of_classes))

for i in range(N):
    Y[i,labels[i]-1] = 1
    
X = np.array(X) 
Y = np.array(Y)

In [822]:
X

array([[ 9.21323266, 11.82445528, 16.69098092, 19.56967227],
       [16.10344524, 23.89625138, 32.17012852, 39.75094393],
       [31.28930267, 47.86461292, 64.21983155, 79.74692811],
       ...,
       [ 7.90082578, 11.96311621, 16.30620007, 19.39478341],
       [15.07843625, 24.22942082, 31.61172694, 39.75461333],
       [32.27519136, 48.00916618, 64.0824622 , 80.59215177]])

In [823]:
Y

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

### Number of layers = all layers except the input layers

### 2 Layer NN -> 1 Hidden Layer and 1 Output Layer

### First work with 2 Layer NN then we will extend this NN


In [824]:
L = 2 # number of layers

k = [4,Y.shape[1]]

W = []

for i in range(L):
    if(i==0):
        W.append(np.random.uniform(-1,1,(k[i],X.shape[1]+1)))
    else:
        W.append(np.random.uniform(-1,1,(k[i],k[i-1]+1)))

In [825]:
def sigmoid(z):
    return 1/(1+np.exp(z))

def derivative(z):
    return sigmoid(z)*(1-sigmoid(z))

In [826]:
W # random weights

[array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
        [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
        [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
        [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]),
 array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
        [ 0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917],
        [ 0.2150897 , -0.65895175, -0.86989681,  0.89777107,  0.93126407],
        [ 0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]])]

In [827]:
for each in range(500):
    accuracy = 0
    for i in range(N):
        #forward propagation
        y_r = np.hstack((np.ones(1),X[i]))
        y = []
        y.append(y_r)
        v = []
        for r in range(L):
            v_r = np.zeros(k[r])
            for j in range(k[r]):
                v_r[j] = np.dot(W[r][j],y_r)
            v.append(v_r)
            y_r = np.hstack((np.ones(1),sigmoid(v_r)))
            y.append(y_r)
        
        # cost calculation
        #J += np.sum((y[L][1:]-Y[i])**2,axis=0)
            
        #back propagation
        
        if(np.argmax(y[L][1:])==np.argmax(Y[i])):
            accuracy += 1
        
        delta_L = (Y[i]-y[L][1:])*derivative(v[L-1])
        
        delta = []

        for r in range(L):
            if(r==L-1):
                delta.append(delta_L)
            else:
                delta.append(np.zeros(k[r]))

        for r in range(L-1):
            delta_r = []
            for j in range(k[L-r-2]):
                total = 0
                for p in range(k[L-r-1]):
                    total += delta[L-r-1][p]*W[L-r-1][p][j]
                delta_r.append(total*derivative(v[L-r-2][j]))

            delta[L-r-2] = np.array(delta_r)


        y = y[:-1]

        for r in range(L):
            for j in range(k[r]):
                W[r][j] = W[r][j] - 0.03*(delta[r][j]*y[r])

    print('Epoch: ',each,(accuracy/N)*100)

Epoch:  0 21.6
Epoch:  1 22.6
Epoch:  2 23.400000000000002
Epoch:  3 24.0
Epoch:  4 27.0
Epoch:  5 30.2
Epoch:  6 30.0
Epoch:  7 31.8
Epoch:  8 30.8
Epoch:  9 31.8
Epoch:  10 30.599999999999998
Epoch:  11 30.4
Epoch:  12 35.6
Epoch:  13 34.2
Epoch:  14 33.0
Epoch:  15 33.2
Epoch:  16 34.599999999999994
Epoch:  17 35.199999999999996
Epoch:  18 35.8
Epoch:  19 36.6
Epoch:  20 37.2
Epoch:  21 36.199999999999996
Epoch:  22 38.2
Epoch:  23 39.0
Epoch:  24 39.0
Epoch:  25 38.0
Epoch:  26 38.0
Epoch:  27 41.0
Epoch:  28 40.400000000000006
Epoch:  29 41.199999999999996
Epoch:  30 40.400000000000006
Epoch:  31 40.6
Epoch:  32 38.800000000000004
Epoch:  33 41.8
Epoch:  34 40.0
Epoch:  35 44.0
Epoch:  36 40.6
Epoch:  37 39.4
Epoch:  38 43.2
Epoch:  39 40.400000000000006
Epoch:  40 41.6
Epoch:  41 44.4
Epoch:  42 38.2
Epoch:  43 43.4
Epoch:  44 43.4
Epoch:  45 41.8
Epoch:  46 39.800000000000004
Epoch:  47 40.8
Epoch:  48 44.0
Epoch:  49 44.4
Epoch:  50 43.0
Epoch:  51 41.6
Epoch:  52 42.0
Epoch:  

Epoch:  396 54.2
Epoch:  397 54.400000000000006
Epoch:  398 55.2
Epoch:  399 56.2
Epoch:  400 54.800000000000004
Epoch:  401 57.4
Epoch:  402 54.400000000000006
Epoch:  403 56.99999999999999
Epoch:  404 55.2
Epoch:  405 54.0
Epoch:  406 56.2
Epoch:  407 55.400000000000006
Epoch:  408 56.8
Epoch:  409 55.800000000000004
Epoch:  410 55.60000000000001
Epoch:  411 56.39999999999999
Epoch:  412 56.99999999999999
Epoch:  413 56.99999999999999
Epoch:  414 56.8
Epoch:  415 55.00000000000001
Epoch:  416 54.400000000000006
Epoch:  417 54.400000000000006
Epoch:  418 55.00000000000001
Epoch:  419 54.2
Epoch:  420 56.8
Epoch:  421 55.00000000000001
Epoch:  422 54.800000000000004
Epoch:  423 55.60000000000001
Epoch:  424 55.400000000000006
Epoch:  425 55.00000000000001
Epoch:  426 55.400000000000006
Epoch:  427 56.8
Epoch:  428 54.6
Epoch:  429 55.2
Epoch:  430 55.00000000000001
Epoch:  431 54.800000000000004
Epoch:  432 54.2
Epoch:  433 54.6
Epoch:  434 55.60000000000001
Epoch:  435 54.2
Epoch:  43

In [828]:
W

[array([[-11.61058312,   0.79249341,   0.81250274,   0.12296536,
          -0.70895373],
        [ -0.6864924 ,  -0.87140241,   0.75035727,   0.2265067 ,
           0.44631292],
        [ -0.72251684,   0.9887698 ,   0.69097357,  -0.50693729,
          -0.5033526 ],
        [ -0.63328412,  -0.3922511 ,   0.0483987 ,  -0.13759784,
          -0.41939261]]),
 array([[ 2.65847100e+00, -9.00989144e+00, -4.15989072e-01,
          3.72933173e+00,  2.34704568e+00],
        [ 1.65464794e+00, -1.25351157e+00,  2.86269342e-02,
          3.69197064e-03,  1.77099321e-01],
        [-2.75302160e-01,  1.60406098e+00, -8.69818897e-01,
          4.22720359e-02,  4.40838446e-01],
        [ 2.25617004e-01,  6.67647408e+00, -8.04596750e-01,
         -1.44159298e-01, -5.10894601e-01]])]

In [830]:
for i in range(N):
    #forward propagation
    y_r = np.hstack((np.ones(1),X[i]))
    y = []
    y.append(y_r)
    v = []
    for r in range(L):
        v_r = np.zeros(k[r])
        for j in range(k[r]):
            v_r[j] = np.dot(W[r][j],y_r)
        v.append(v_r)
        y_r = np.hstack((np.ones(1),sigmoid(v_r)))
        y.append(y_r)
    
    print(np.argmax(y[L][1:]))

0
0
3
2
3
0
0
2
0
2
2
2
2
0
0
0
0
2
3
0
0
3
0
0
3
0
3
0
0
1
1
2
1
0
0
3
0
2
3
0
3
0
3
3
0
1
3
0
3
2
3
3
0
0
2
0
3
2
2
0
0
3
0
3
0
3
0
3
3
2
0
0
1
3
0
3
3
0
2
0
0
3
2
0
2
0
0
0
0
0
0
0
0
0
2
0
2
0
0
0
0
2
0
1
3
1
2
3
3
2
3
0
0
3
3
0
2
0
0
0
0
0
0
0
3
1
3
0
0
3
3
2
3
2
2
3
3
3
2
0
0
3
0
0
0
2
0
2
3
0
0
0
3
2
0
0
2
3
3
3
0
2
3
0
3
3
0
0
0
3
0
0
2
0
0
3
2
0
3
0
0
2
0
0
2
3
3
0
0
0
3
0
0
2
0
3
3
3
2
2
3
0
0
0
3
2
0
0
1
0
0
0
0
0
0
0
3
0
0
2
3
0
0
0
0
2
0
2
2
2
2
3
1
3
0
0
0
1
0
1
1
2
2
2
0
0
3
0
0
0
2
3
3
0
3
1
0
2
0
2
2
0
0
3
0
3
0
0
3
3
3
3
0
1
3
2
3
0
3
3
2
0
0
1
0
0
3
2
0
0
0
3
3
2
3
0
0
0
0
3
1
2
3
2
2
0
2
0
0
1
2
0
0
2
2
0
0
0
0
0
0
2
0
0
3
3
0
0
3
0
3
3
2
0
0
3
2
0
2
0
2
3
3
0
2
0
3
0
0
3
0
2
3
2
0
0
0
0
2
0
3
1
0
2
3
3
1
0
0
2
0
0
3
3
3
3
0
2
1
3
2
0
2
3
1
0
3
0
1
0
0
3
0
3
2
0
0
0
3
0
2
2
2
3
0
0
3
3
0
0
2
0
2
0
0
2
2
3
0
2
0
3
2
3
0
3
2
1
0
0
3
0
0
2
3
2
3
0
0
2
0
2
0
3
0
0
3
2
0
0
2
1
0
0
3
0
0
0
0
3
0
2
0
0
3
0
0
2
0
0
2
3
1
0
0
0
0
3
2
0
2
0
3
0
2
2
2
0
0
0
0
2
0
3
2
3
2
0
0
3
